In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phishing-email-dataset/SpamAssasin.csv
/kaggle/input/phishing-email-dataset/Nazario.csv
/kaggle/input/phishing-email-dataset/Nigerian_Fraud.csv
/kaggle/input/phishing-email-dataset/CEAS_08.csv
/kaggle/input/phishing-email-dataset/Enron.csv
/kaggle/input/phishing-email-dataset/Ling.csv
/kaggle/input/phishing-email-dataset/phishing_email.csv


In [2]:
# Load the dataset
df = pd.read_csv("/kaggle/input/phishing-email-dataset/phishing_email.csv")

In [3]:
df.head()

,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


In [4]:
print("\nShape = ",df.shape)
print("\nUnique Label = ",df["label"].unique())
print("\nNA Values = \n",df.isna().sum())


Shape =  (82486, 2)

Unique Label =  [0 1]

NA Values = 
 text_combined    0
label            0
dtype: int64


In [5]:
# Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["text_combined"], df["label"], test_size=0.2, random_state=41)

In [6]:
print("X_train shape = ",X_train.shape)
print("y_train shape = ",y_train.shape)
print("X_test shape = ",X_train.shape)
print("y_test shape = ",X_train.shape)

X_train shape =  (65988,)
y_train shape =  (65988,)
X_test shape =  (65988,)
y_test shape =  (65988,)


In [7]:
import torch
import torchvision
from transformers import BertTokenizer, BertModel

2025-06-26 16:12:43.713117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750954363.934219      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750954363.996889      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
def tokenize_function(texts, labels):
    encoding = tokenizer(texts, padding="max_length", max_length=128, truncation=True, return_tensors="pt")
    return encoding["input_ids"], encoding["attention_mask"], torch.tensor(labels, dtype=torch.float)

In [10]:
# tokenize_function(["Hello, How are you!"],[0])

In [11]:
train_input_ids, train_attention_mask, train_labels = tokenize_function(X_train.values.tolist(), y_train.values.tolist())
val_input_ids, val_attention_mask, val_labels = tokenize_function(X_test.values.tolist(), y_test.values.tolist())


In [12]:
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = torch.utils.data.TensorDataset(val_input_ids, val_attention_mask, val_labels)

In [13]:
batch_size=64

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [14]:
# Load the pretrained model

bert = BertModel.from_pretrained("bert-base-uncased")
bert.config.hidden_size

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

768

In [21]:
# Define the model
import torch.nn as nn

class SentimentClassifier(nn.Module):

    def __init__(self):

        super(SentimentClassifier, self).__init__()

        self.bert = BertModel.from_pretrained("bert-base-uncased")

        for param in self.bert.parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask):

        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sentence_embedding = bert_output.last_hidden_state[:,0,:]
        return self.classifier(sentence_embedding)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device : ",device)

Device :  cuda


In [28]:
model=SentimentClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [29]:
epochs = 2

for epoch in range(epochs):
    model.train()
    total_train_loss = 0.0
    for batch, (input_ids, attention_mask, labels) in enumerate(train_dataloader):

        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask.to(device)).squeeze()
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    print(f"Epoch: {epoch+1}/{epochs}, Loss: {total_train_loss / len(train_dataloader)}")

Epoch: 1/2, Loss: 0.2135836694158556
Epoch: 2/2, Loss: 0.14778857860468733


In [39]:
# Test the model

model.eval()

total_val_loss  = 0
correct_predictions  = 0

with torch.no_grad():

    for input_ids, attention_mask, labels in val_dataloader:

        outputs = model(input_ids.to(device), attention_mask.to(device)).squeeze()

        loss = criterion(outputs, labels.to(device).view_as(outputs))

        total_val_loss += loss.item()

        preds = (outputs > 0.5).float()
        # print(preds)
        # print(torch.sum(preds.cpu() == labels.cpu()))
        correct_predictions += torch.sum(preds.cpu() == labels.cpu())

avg_val_loss = total_val_loss / len(val_dataloader)
val_accuracy = correct_predictions.double() / len(val_dataset)
print(f"Validation Loss : {avg_val_loss:.4f}, Validation Accuracy : {val_accuracy:.4f}")

Validation Loss : 0.1045, Validation Accuracy : 0.9591


In [44]:
def predict(model, text, max_length=128):

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # Tokenize input text
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    model.eval()
    with torch.no_grad():
        output = model(input_ids.to(device), attention_mask.to(device)).squeeze()
        prediction = (output > 0.5).float().item()
        return 'spam' if prediction == 1 else "ham"

In [45]:
predict(model, "Hello, You've won 1000$ free prize.")

'spam'

In [47]:
predict(model, "Hello, Please click on below link to avail your ")

'spam'

In [48]:
predict(model, "Hello Sam - When can we expect the result?")

'ham'